# 📊 Telecom X - Análisis de Evasión de Clientes

## Introducción

A continuación, se presenta un análisis sobre la problemática de la fuga de clientes en la empresa Telecom X. Este estudio ha sido elaborado con el máximo rigor profesional, aplicando los conocimientos y las técnicas adquiridas durante mi formación en Alura Latam.

Este notebook documenta cada paso de mi proceso, desde la configuración inicial del proyecto y la carga de datos, hasta la limpieza profunda, el análisis exploratorio y, finalmente, la síntesis de insights accionables. Para darle un enfoque didáctico y para transparentar mi razonamiento aplicado, cada una de mis acciones/decisiones; las presentare detallando su: 

1) **Objetivo**, 
2) **Método** y 
3) **Justificación.**

El objetivo final es proporcionar al equipo de negocio recomendaciones estratégicas basadas en evidencia y, al mismo tiempo, entregar al equipo de Data Science un dataset preparado para la construcción de modelos predictivos.

## 📌 1. Configuración y Extracción (**E**)

### 🎯 1.1. Importación de Librerías

**Objetivo**: Cargar todas las herramientas (librerías de Python) que necesitaremos para el análisis.

**Método**: Importaré las librerías estándar para manipulación de datos, operaciones numéricas y visualización.

**Justificación**:
* *Pandas*: Es la herramienta fundamental en Python para la manipulación y el análisis de datos tabulares.
* *NumPy*: Proporciona soporte para operaciones numéricas eficientes.
* *Plotly*: Mi librería de visualización principal para generar gráficos interactivos, lo cual es invaluable durante la fase de exploración.
* *Seaborn/Matplotlib*: Excelentes para ciertos tipos de gráficos estáticos, como matrices de correlación (heatmaps).

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# import seaborn as sns # al final no se utilizo pero lo agregue al inicio por si lo llegaba a ocupar
# import matplotlib.pyplot as plt # igual no se utilizo

# Configuraciones adicionales para una mejor visualización
pd.set_option('display.max_columns', None)
print("Librerías importadas exitosamente.")

Librerías importadas exitosamente.


### 🎯 1.2. Extracción de Datos y Perfilamiento Inicial

**Objetivo**: Cargar los datos desde la fuente y realizar una primera validación de su estructura y contenido. 

**Método**: Utilizaré la función read_json de Pandas para cargar los datos directamente desde la URL. Inmediatamente después verificaré las dimensiones del dataset, los tipos de datos y la presencia inicial de valores nulos.

**Justificación**: *Nunca debemos asumir que los datos están listos para el análisis.* Este primer vistazo es crucial para detectar problemas estructurales de inmediato. Por ejemplo, los datos en formato JSON a menudo contienen estructuras anidadas que deben ser aplanadas (**"normalizadas"**) antes de que podamos analizarlas eficazmente.

In [2]:
# URL de los datos en formato JSON proporcionado por Alura Latam
url = 'https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/main/TelecomX_Data.json'

# Se obtienen los datos y se convierten a un DataFrame de Pandas
try:
    df_raw = pd.read_json(url)
    print("¡Datos cargados exitosamente!")
except Exception as e:
    print(f"Error al cargar los datos: {e}")

# Visualizamos las primeras 5 filas para una primera impresión
df_raw.head()

¡Datos cargados exitosamente!


,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


In [3]:
# Realizando el perfilamiento inicial
print("Dimensiones del DataFrame (Filas, Columnas):")
print(df_raw.shape)
print("\n" + "="*50 + "\n") # (barra) solo estilizando
print("Información general y tipos de datos:")
df_raw.info()

Dimensiones del DataFrame (Filas, Columnas):
(7267, 6)


Información general y tipos de datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   customerID  7267 non-null   object
 1   Churn       7267 non-null   object
 2   customer    7267 non-null   object
 3   phone       7267 non-null   object
 4   internet    7267 non-null   object
 5   account     7267 non-null   object
dtypes: object(6)
memory usage: 340.8+ KB


#### 💡 Primeras Observaciones

Basado en el perfilamiento inicial, he descubierto lo siguiente:

1. **Dimensiones**: El dataset contiene **7267 filas** y **6 columnas**. Esto es muy sospechoso. porque supongo que un análisis de clientes debería tener muchas más de cuatro características.

2. **Estructura Anidada**: Al ver mas detalladamente, noto que todas las columnas son de tipo object. La vista previa con .head() sugiere que contienen estructuras JSON anidadas. Asi que mi primer paso de transformación deberá ser **normalizar** o aplanar estos datos para que cada característica tenga su propia columna.

3. **Valores Nulos**: `df.info()` indica que no hay valores nulos. Sin embargo, no debo hacer este tipo de conclusiones aun, ya que los valores nulos podrían estar ocultos dentro de las estructuras JSON anidadas. Deberé reevaluar la presencia de nulos después de **normalizar**.

## 🔧 2. Transformación y Limpieza (**T**)

### 🎯 2.1. Normalización de Datos (Aplanamiento)

**Objetivo**: Transformar el dataset de su formato anidado a un formato tabular plano, donde cada pieza de información tenga su propia columna.

**Método**: Utilizaré la función `pd.json_normalize()` de Pandas. La aplicaré a cada una de las columnas complejas (`customer`, `account`, `phone`, `internet`) y luego uniré los resultados.

**Justificación**: Sin este paso, no podríamos calcular correlaciones, crear visualizaciones efectivas.

In [4]:
from pandas import json_normalize

# Normalizar las columnas anidadas en sus propios DataFrames
df_customer = json_normalize(df_raw['customer'])
df_account = json_normalize(df_raw['account'])
df_phone = json_normalize(df_raw['phone'])
df_internet = json_normalize(df_raw['internet'])

# Unir los nuevos DataFrames con las columnas originales que queremos conservar
df_normalized = pd.concat([
    df_raw[['customerID', 'Churn']],
    df_customer,
    df_account,
    df_phone,
    df_internet
], axis=1)

print("Datos normalizados exitosamente.")
print("Nuevas dimensiones del DataFrame (Filas, Columnas):", df_normalized.shape)
df_normalized.head()

Datos normalizados exitosamente.
Nuevas dimensiones del DataFrame (Filas, Columnas): (7267, 21)


,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,0002-ORFBO,No,Female,0,Yes,Yes,9,One year,Yes,Mailed check,65.6,593.3,Yes,No,DSL,No,Yes,No,Yes,Yes,No
1,0003-MKNFE,No,Male,0,No,No,9,Month-to-month,No,Mailed check,59.9,542.4,Yes,Yes,DSL,No,No,No,No,No,Yes
2,0004-TLHLJ,Yes,Male,0,No,No,4,Month-to-month,Yes,Electronic check,73.9,280.85,Yes,No,Fiber optic,No,No,Yes,No,No,No
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Month-to-month,Yes,Electronic check,98.0,1237.85,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,Yes
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Month-to-month,Yes,Mailed check,83.9,267.4,Yes,No,Fiber optic,No,No,No,Yes,Yes,No


### 🎯 2.2. Segundo Perfilamiento

**Objetivo**: Ahora con los datos aplanados, debo realizar una nueva y más profunda verificación de su calidad.

**Método**: Volveré a usar `.info()` y, de manera crucial, `.isnull().sum()` para obtener un recuento exacto de los valores faltantes por columna.

**Justificación**: Al haber **nomralizado** los datos, ahora puedo ver ampliamente posibles problemas que antes estaban ocultos.

In [5]:
# Obtener información detallada del nuevo DataFrame
df_normalized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7267 non-null   object 
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   object 
 5   Dependents        7267 non-null   object 
 6   tenure            7267 non-null   int64  
 7   Contract          7267 non-null   object 
 8   PaperlessBilling  7267 non-null   object 
 9   PaymentMethod     7267 non-null   object 
 10  Charges.Monthly   7267 non-null   float64
 11  Charges.Total     7267 non-null   object 
 12  PhoneService      7267 non-null   object 
 13  MultipleLines     7267 non-null   object 
 14  InternetService   7267 non-null   object 
 15  OnlineSecurity    7267 non-null   object 
 16  OnlineBackup      7267 non-null   object 


#### ✋ Alto

Aunque el análisis preliminar no arroja valores nulos (NaN), la inspección de los tipos de datos podría revelar una inconsistencia. Esta situación sugiere la presencia de valores vacíos que fueron interpretados como cadenas de texto ("") en lugar de nulos, lo cual explicaría el tipado incorrecto en las columnas afectadas. Aquí me enfrento a la primera decisión de abordaje, ya que me imagino varias maneras de dar solución.

##### 🎯 2.2.1. Manejo de Inconsistencias: Strings Vacíos

**Objetivo**: Identificar y estandarizar todos los valores que vacíos o representados como "", para que sean reconocidos como valores NaN.

**Método**: Utilizaré el método `.replace()` de Pandas con una **expresión regular** `(r'^\s*$')` para buscar en todo el DataFrame cualquier celda que contenga únicamente espacios en blanco (o que esté completamente vacía) y reemplazarla con `np.nan`.

**Justificación**: Al convertir todos los tipos de "vacíos" a NaN, podre utilizar de manera fiable las funciones de Pandas como `.isnull().sum()`, `.fillna()` y `.dropna()` para manejar los datos faltantes.

In [6]:
# Crear una copia para el proceso de limpieza
df_clean = df_normalized.copy()

# Reemplazar strings vacíos o con solo espacios en blanco por NaN en todo el DataFrame
df_clean.replace(r'^\s*$', np.nan, regex=True, inplace=True)

print("Strings vacíos reemplazados por NaN.")

Strings vacíos reemplazados por NaN.


### 🎯 2.3. Tercer Perfilamiento

**Objetivo**: Con los valores vacíos estandarizados, vamos a re-evaluar la calidad de los datos.

**Método**: Ejecutaré `.isnull().sum()`.

**Justificación**: Se espera que el resultado de este comando nos dará ahora sí el panorama real de los datos faltantes.

In [7]:
# Verificar de nuevo la cantidad de valores nulos por columna
print("Conteo de valores nulos por columna:")
df_clean.isnull().sum()

Conteo de valores nulos por columna:


customerID            0
Churn               224
gender                0
SeniorCitizen         0
Partner               0
Dependents            0
tenure                0
Contract              0
PaperlessBilling      0
PaymentMethod         0
Charges.Monthly       0
Charges.Total        11
PhoneService          0
MultipleLines         0
InternetService       0
OnlineSecurity        0
OnlineBackup          0
DeviceProtection      0
TechSupport           0
StreamingTV           0
StreamingMovies       0
dtype: int64

#### 🎯 2.3.1. Investigación de Nulos en la Columna Churn

**Objetivo**: Perfilar los 224 clientes con datos de Churn nulos para determinar si constituyen un subgrupo con características específicas. El fin es tomar una decisión informada sobre cómo manejarlos, en lugar de simplemente eliminarlos.

**Método**: Creare dos DataFrames temporales: uno con los 224 clientes con Churn nulo (`df_null_churn`) y otro con el resto de los clientes (`df_known_churn`). Realizare un análisis comparativo entre ambos grupos. Específicamente, comparare:
* Las estadísticas descriptivas de las variables numéricas.
* La distribución de variables categóricas clave.

**Justificación**: La eliminación de datos es el último recurso. Si estos 224 clientes pertenecen a un grupo específico (por ejemplo, todos son clientes corporativos, todos tienen un tipo de contrato especial, o todos son registros de prueba), eliminarlos podría sesgar mi análisis general. 

In [8]:
# 1. Crear los dos DataFrames para la comparación
df_null_churn = df_clean[df_clean['Churn'].isnull()]
df_known_churn = df_clean[df_clean['Churn'].notnull()]

print("--- Perfil de Clientes con Churn NULO ---")
print(f"Número de clientes: {df_null_churn.shape[0]}")
print("\nAnálisis Numérico:")
print(df_null_churn[['tenure', 'Charges.Monthly', 'Charges.Total']].describe())
print("\nAnálisis Categórico (Tipo de Contrato):")
print(df_null_churn['Contract'].value_counts(normalize=True))

print("\n" + "="*50 + "\n") # barra de apoyo visual

print("--- Perfil de Clientes con Churn CONOCIDO ---")
print(f"Número de clientes: {df_known_churn.shape[0]}")
print("\nAnálisis Numérico:")
print(df_known_churn[['tenure', 'Charges.Monthly', 'Charges.Total']].describe())
print("\nAnálisis Categórico (Tipo de Contrato):")
print(df_known_churn['Contract'].value_counts(normalize=True))

--- Perfil de Clientes con Churn NULO ---
Número de clientes: 224

Análisis Numérico:
           tenure  Charges.Monthly
count  224.000000       224.000000
mean    31.571429        63.412277
std     24.998552        31.388712
min      1.000000        18.750000
25%      7.000000        28.425000
50%     29.000000        69.100000
75%     56.000000        90.412500
max     72.000000       115.550000

Análisis Categórico (Tipo de Contrato):
Contract
Month-to-month    0.580357
Two year          0.214286
One year          0.205357
Name: proportion, dtype: float64


--- Perfil de Clientes con Churn CONOCIDO ---
Número de clientes: 7043

Análisis Numérico:
            tenure  Charges.Monthly
count  7043.000000      7043.000000
mean     32.371149        64.761692
std      24.559481        30.090047
min       0.000000        18.250000
25%       9.000000        35.500000
50%      29.000000        70.350000
75%      55.000000        89.850000
max      72.000000       118.750000

Análisis Categóri

💡 Interpretación de la Evidencia

Al comparar los dos perfiles, observo:

**Variables Numéricas**: La antigüedad media (`tenure`), los cargos mensuales (`Charges.Monthly`) y los cargos totales (`Charges.Total`) son estadísticamente muy similares en ambos grupos. Por ejemplo, la antigüedad media es de ~32 meses en ambos casos, y los cargos mensuales promedian ~$64.

**Variables Categóricas**: La distribución de los tipos de contrato es casi idéntica. En ambos grupos, el ~55% de los clientes tienen contratos mes a mes, el ~24% tienen contratos de dos años y el ~21% de un año.

**Conclusión**: El análisis demuestra que los 224 clientes con `Churn` nulo **no forman un grupo distinto**. Sus características demográficas, de uso y de contratación **son representativas de la población general de clientes**. Esto me sugiere que la falta de datos de `Churn` es un **problema aleatorio** (posiblemente un error de registro o un fallo en la recolección de datos), y no se debe a que estos clientes pertenezcan a una categoría especial. Mi teoria fue erronea, pero ahora puedo avanzar con más confianza.

#### 🎯 2.3.2. Acción Final sobre Nulos en `Churn`

**Objetivo**: Tomar una acción final y justificada sobre los 224 registros con `Churn` nulo.

**Método**: AHora que confirme que estos registros no representan un subgrupo único y que la información faltante es la variable objetivo misma, procedo con la eliminación de estas filas.

**Justificación**: Mi investigación ha descartado la posibilidad de que los nulos en `Churn` identifiquen a un segmento de clientes particular. Su eliminación es, por tanto, la acción metodológicamente más sólida, asegurando que el análisis se base únicamente en datos completos y verificables.

In [9]:
print(f"Número de filas ANTES de la eliminación: {df_clean.shape[0]}")

# Eliminamos las filas donde 'Churn' es nulo
df_clean.dropna(subset=['Churn'], inplace=True)

print(f"Número de filas DESPUÉS de la eliminación: {df_clean.shape[0]}")

Número de filas ANTES de la eliminación: 7267
Número de filas DESPUÉS de la eliminación: 7043


#### 🎯 2.3.3. Investigación de Nulos en Charges.Total

**Objetivo**: Investigar las 11 filas restantes donde la columna Charges.Total es nula.

**Método**: Mi hipótesis de que estos nulos están directamente relacionados con la antigüedad (tenure) del cliente y para verificarlo, filtrare el DataFrame `df_clean` para aislar y examinar únicamente estas 11 filas.

**Justificación**: En los datos de facturación por suscripción, es común que los valores totales faltantes correspondan a **clientes nuevos** que aún no han cumplido su primer ciclo de facturación, por lo que si confirmo esta relación me permitira aplicar una imputación basada en la lógica del negocio en lugar de una suposición estadística (como la media o la mediana), lo cual es considerablemente más preciso.

In [10]:
# Filtramos el dataframe para ver las filas con Charges.Total nulo y seleccionamos columnas clave para el análisis.
df_clean[df_clean['Charges.Total'].isnull()][['tenure', 'Charges.Monthly', 'Charges.Total']]

,tenure,Charges.Monthly,Charges.Total
975,0,56.05,NaN
1775,0,20.00,NaN
1955,0,61.90,NaN
2075,0,19.70,NaN
2232,0,20.25,NaN
2308,0,25.35,NaN
2930,0,73.35,NaN
3134,0,25.75,NaN
3203,0,52.55,NaN
4169,0,80.85,NaN


💡 Interpretación de la Evidencia

***¡Eureka!*** en las 11 filas donde `Charges.Total` es nulo, el valor en la columna `tenure` (**antigüedad**) es 0. Por lo tanto mi hipótesis queda confirmada. estos registros **pertenecen a clientes nuevos** que, al tener 0 meses de antigüedad, lógicamente aún no han acumulado ningún cargo total, asi que El valor nulo representa un **0**.

#### 🎯 2.3.4. Acción Final sobre Nulos en Charges.Total

**Objetivo**: Imputar los 11 valores nulos en `Charges.Total` con un valor que refleje la situación comercial de estos clientes.

**Método**: Se rellenarán con el número 0.0 utilizando el método `.fillna()`, para después, convertir toda la columna a tipo float.

**Justificación**: Habiendo probado que estos nulos corresponden a clientes con 0 meses de antigüedad, imputarlos con 0.0 es la única acción lógicamente correcta, ya que esto reflejaría su estado de facturación real.



In [11]:
# Rellenamos los valores nulos en 'Charges.Total' con 0
df_clean['Charges.Total'] = df_clean['Charges.Total'].fillna(0)

# Verificamos que ya no quedan nulos en la columna
print(f"Nulos restantes en 'Charges.Total': {df_clean['Charges.Total'].isnull().sum()}")

# Ahora convertimos la columna a un tipo de dato numérico
df_clean['Charges.Total'] = pd.to_numeric(df_clean['Charges.Total'])

print(f"Nuevo tipo de dato de 'Charges.Total': {df_clean['Charges.Total'].dtype}")

Nulos restantes en 'Charges.Total': 0
Nuevo tipo de dato de 'Charges.Total': float64


### 🎯 2.4 Optimización de Tipos de Datos

#### 🎯 2.4.1 Exploración de Columnas de Tipo Object

**Objetivo**: Investigar sistemáticamente todas las columnas de tipo object restantes para clasificar su contenido y determinar el tipo de dato más óptimo para cada una.

**Método**: Creare un bucle que itere a través de todas las columnas del DataFrame `df_clean`. Para cada columna de tipo `object`, se imprimirá su nombre, el número de valores únicos que contiene y una muestra de esos valores únicos.

**Justificación**: Este análisis exploratorio es crucial para tomar decisiones informadas sobre la ***optimización de tipos de datos***.

In [12]:
print("--- Análisis de Cardinalidad de Columnas 'object' ---")

# Iteramos sobre cada columna del DataFrame
for col in df_clean.columns:
    # Verificamos si la columna es de tipo 'object'
    if df_clean[col].dtype == 'object':
        unique_count = df_clean[col].nunique()
        print(f"\nColumna: '{col}'")
        print(f"  - Número de valores únicos: {unique_count}")
        print(f"  - Muestra de valores: {df_clean[col].unique()[:5]}") # Mostramos hasta 5 valores únicos como ejemplo

--- Análisis de Cardinalidad de Columnas 'object' ---

Columna: 'customerID'
  - Número de valores únicos: 7043
  - Muestra de valores: ['0002-ORFBO' '0003-MKNFE' '0004-TLHLJ' '0011-IGKFF' '0013-EXCHZ']

Columna: 'Churn'
  - Número de valores únicos: 2
  - Muestra de valores: ['No' 'Yes']

Columna: 'gender'
  - Número de valores únicos: 2
  - Muestra de valores: ['Female' 'Male']

Columna: 'Partner'
  - Número de valores únicos: 2
  - Muestra de valores: ['Yes' 'No']

Columna: 'Dependents'
  - Número de valores únicos: 2
  - Muestra de valores: ['Yes' 'No']

Columna: 'Contract'
  - Número de valores únicos: 3
  - Muestra de valores: ['One year' 'Month-to-month' 'Two year']

Columna: 'PaperlessBilling'
  - Número de valores únicos: 2
  - Muestra de valores: ['Yes' 'No']

Columna: 'PaymentMethod'
  - Número de valores únicos: 4
  - Muestra de valores: ['Mailed check' 'Electronic check' 'Credit card (automatic)'
 'Bank transfer (automatic)']

Columna: 'PhoneService'
  - Número de valores 

#### 🎯 2.4.2 Transformación de Tipos de Datos

**Objetivo**: Convertir las columnas de tipo object a tipos de datos más específicos y eficientes, basándonos en la clasificación obtenida en el paso exploratorio anterior.

**Método**:

* ***Variables Binarias***: Identificare explícitamente todas las columnas que contienen únicamente valores `'Yes'/'No'` (o `'Male'/'Female'`) y se mapearán a `1` y `0` respectivamente. Esto incluye la variable objetivo Churn.

* ***Variables Categóricas***: Las columnas con un número limitado de categorías de texto (ej. `'Contract'`, `'InternetService'`, y las que incluyen `'No internet service'`) se convertirán directamente al tipo category de Pandas.

* ***Identificadores***: La columna customerID, al ser un identificador único, la mantendre como tipo object.

**Justificación**: Esta transformación esta justificada por la exploración previa, y se espera reducir drásticamente el uso de memoria, la conversión de variables binarias a números (`0` y `1`) es un paso de preprocesamiento estándar y esencial para el modelado, por otro lado el tipo category es mucho más eficiente en memoria y rendimiento que object para columnas con texto repetido. Segun la documentacion, internamente `Pandas` almacena las categorías una sola vez y luego usa códigos enteros para referenciarse a ellas, lo que acelera las operaciones de agrupación y unión.

In [13]:
# Hacemos una copia para trabajar de forma segura
df_final = df_clean.copy()

# 1. Mapeo de columnas estrictamente binarias a 1/0
binary_cols_yes_no = ['Partner', 'Dependents', 'PaperlessBilling', 'PhoneService']
for col in binary_cols_yes_no:
    df_final[col] = df_final[col].map({'Yes': 1, 'No': 0})

# Mapeo específico para 'gender' y la variable objetivo 'Churn'
df_final['gender'] = df_final['gender'].map({'Male': 0, 'Female': 1})
df_final['Churn'] = df_final['Churn'].map({'Yes': 1, 'No': 0})

# 2. Conversión a tipo 'category' para columnas con múltiples categorías de texto.
# Este enfoque es el correcto para manejar valores como 'No phone service' o 
# 'No internet service' sin generar valores nulos.
categorical_cols = [
    'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
    'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
    'Contract', 'PaymentMethod'
]
for col in categorical_cols:
    df_final[col] = df_final[col].astype('category')

# Verificamos el resultado. Ahora no deberían aparecer nulos nuevos.
print("Tipos de datos optimizados. Verificando el nuevo estado del DataFrame:")
df_final.info()

Tipos de datos optimizados. Verificando el nuevo estado del DataFrame:
<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   customerID        7043 non-null   object  
 1   Churn             7043 non-null   int64   
 2   gender            7043 non-null   int64   
 3   SeniorCitizen     7043 non-null   int64   
 4   Partner           7043 non-null   int64   
 5   Dependents        7043 non-null   int64   
 6   tenure            7043 non-null   int64   
 7   Contract          7043 non-null   category
 8   PaperlessBilling  7043 non-null   int64   
 9   PaymentMethod     7043 non-null   category
 10  Charges.Monthly   7043 non-null   float64 
 11  Charges.Total     7043 non-null   float64 
 12  PhoneService      7043 non-null   int64   
 13  MultipleLines     7043 non-null   category
 14  InternetService   7043 non-null   category
 15  Online

### 🎯 2.5. Creación de Nuevas Características (Feature Engineering)

**Objetivo**: Crear la columna `Cuentas_Diarias` para obtener una métrica de gasto más granular que los cargos mensuales.

**Método**: Se calculará el cargo diario dividiendo la columna de cargos mensuales (`Charges.Monthly`) entre 30.44, que es el promedio de días en un mes. El resultado se redondeará a 2 decimales.

**Justificación**: Esta nueva característica normaliza el gasto en una escala diaria, por lo que nos proporciona una variable adicional que puede tener una correlación diferente con la evasión (`Churn`) y nos da una perspectiva más detallada para el análisis.

In [14]:
# Calculamos los cargos diarios
avg_days_in_month = 30.44
df_final['Cuentas_Diarias'] = (df_final['Charges.Monthly'] / avg_days_in_month).round(2)

# Verificamos la creación de la nueva columna
print("Columna 'Cuentas_Diarias' creada exitosamente. Aquí una muestra:")
df_final[['Charges.Monthly', 'Cuentas_Diarias']].head()

Columna 'Cuentas_Diarias' creada exitosamente. Aquí una muestra:


,Charges.Monthly,Cuentas_Diarias
0,65.6,2.16
1,59.9,1.97
2,73.9,2.43
3,98.0,3.22
4,83.9,2.76


## 📊 3. Carga y Análisis Exploratorio (**L & A**)

Hemos llegado a una etapa emocionante y **muy grafica** 😁, el dataset está limpio, es coherente y está optimizado. En esta fase, dejaremos que los datos nos hablen, mi objetivo sera utilizar estadísticas descriptivas y, sobre todo, visualizaciones para descubrir patrones, identificar relaciones entre variables y empezar a formular hipótesis sobre qué factores influyen en la evasión de clientes.

### 🎯 3.1. Análisis Descriptivo General

**Objetivo**: Obtener un resumen estadístico de las variables numéricas clave en nuestro dataset `df_final`.

**Método**: Utilizare el método `.describe()` de `Pandas`, como ya lo hice anteriormente calcular automáticamente las estadísticas descriptivas más importantes.

**Justificación**: El método `.describe()` es el punto de partida fundamental para cualquier análisis numérico ya que nos proporcionara una visión panorámica instantánea de la distribución de nuestros datos, incluyendo la **tendencia central (media)**, la **mediana**, la **dispersión** y el **rango (mínimo y máximo)**.

In [15]:
# Generamos las estadísticas descriptivas para las columnas numéricas
# Usamos .T para transponer el resultado para una mejor legibilidad
df_final.describe().T

,count,mean,std,min,25%,50%,75%,max
Churn,7043.0,0.265370,0.441561,0.00,0.00,0.00,1.00,1.00
gender,7043.0,0.495244,0.500013,0.00,0.00,0.00,1.00,1.00
SeniorCitizen,7043.0,0.162147,0.368612,0.00,0.00,0.00,0.00,1.00
Partner,7043.0,0.483033,0.499748,0.00,0.00,0.00,1.00,1.00
Dependents,7043.0,0.299588,0.458110,0.00,0.00,0.00,1.00,1.00
tenure,7043.0,32.371149,24.559481,0.00,9.00,29.00,55.00,72.00
PaperlessBilling,7043.0,0.592219,0.491457,0.00,0.00,1.00,1.00,1.00
Charges.Monthly,7043.0,64.761692,30.090047,18.25,35.50,70.35,89.85,118.75
Charges.Total,7043.0,2279.734304,2266.794470,0.00,398.55,1394.55,3786.60,8684.80
PhoneService,7043.0,0.903166,0.295752,0.00,1.00,1.00,1.00,1.00


#### 💡 Primeras Observaciones Numéricas

Del resumen estadístico, puedo extraer algunas ideas iniciales:

* **Antigüedad (tenure)**: El cliente **promedio** tiene una antigüedad de aproximadamente `32 meses` y la distribución es amplia, desde `0 meses` (los que acabamos de corregir) hasta `72 meses` (6 años).

* **Cargos Mensuales (Charges.Monthly)**: El cargo mensual **promedio** es de `$64.76` por lo que considerando que la mediana (el valor del 50%) es de `$70.35`, me sugiere que hay una ligera concentración de clientes con cargos más altos.

* **Cargos Totales (Charges.Total)**: Aquí comprabamos una vez mas el punto de arriba, vemos una gran variabilidad, con un valor máximo de `$8684.80`, un media  de `~$2280` que es significativamente más alta que la mediana `~$1395`, lo cual me indicaria una **distribución sesgada a la derecha**; es decir, hay algunos **clientes con gastos totales muy altos** que tiran del promedio hacia arriba.

### 🎯 3.2. Distribución de la Evasión de Clientes (Churn)

**Objetivo**: Cuantificar y visualizar la proporción exacta de clientes que han cancelado el servicio (`Churn=1`) frente a los que han permanecido (`Churn=0`).

**Método**:

* Utilizare el método `.value_counts()` para obtener el número exacto de clientes en cada categoría y el argumento `normalize=True` para obtener el desglose porcentual.

* Creare un **gráfico de pastel** interactivo utilizando `Plotly` ya que es ideal para mostrar la proporción de cada categoría como parte de un todo.

**Justificación**: Este sera mi punto de partida para el análisis de evasión. Antes de investigar por qué los clientes se van, debo entender la magnitud del problema, y un **gráfico de pastel** es una de las formas más claras y universalmente entendidas de comunicar una **distribución porcentual**, y hacerlo interactivo permitira a los usuarios explorar los datos por sí mismos.

In [16]:
# 1. Calculamos los conteos y porcentajes
churn_counts = df_final['Churn'].value_counts()
churn_percentage = df_final['Churn'].value_counts(normalize=True) * 100

print("Conteo de Evasión de Clientes:")
print(churn_counts)
print("\nPorcentaje de Evasión de Clientes:")
print(round(churn_percentage, 2))

# 2. Creamos la visualización
fig = px.pie(
    names=['Permanecen (No Churn)', 'Evadieron (Churn)'],
    values=churn_counts.values,
    title='Proporción de Evasión de Clientes (Churn)',
    hole=0.3, # Para un efecto de dona (donut chart)
    color_discrete_sequence=px.colors.qualitative.Pastel # Paleta de colores suaves
)

fig.update_traces(
    textinfo='percent+label',
    hovertemplate='<b>%{label}</b><br>Clientes: %{value}<br>Porcentaje: %{percent}'
)

fig.show()

Conteo de Evasión de Clientes:
Churn
0    5174
1    1869
Name: count, dtype: int64

Porcentaje de Evasión de Clientes:
Churn
0    73.46
1    26.54
Name: proportion, dtype: float64


#### 💡 Análisis de la Tasa de Evasión

Tenemo datos claros y directos:

* `1,869` clientes han cancelado su servicio.

* `5,174` clientes permanecen activos.

Esto se traduce en una **tasa de evasión** (churn rate) general del `26.54%`.

En términos de negocio, esto significa que más de uno de cada cuatro clientes ha abandonado la compañía. Esta es una cifra significativa y justifica plenamente la necesidad de este análisis para identificar los factores que impulsan este comportamiento y proponer estrategias de retención.

### 🎯 3.3. Análisis de Evasión por Variables Demográficas

**Objetivo**: Analizar cómo la tasa de evasión (`Churn`) varía entre diferentes segmentos demográficos de clientes, específicamente por género, si es adulto mayor, si tiene pareja y si tiene dependientes.

**Método**:

* Para cada variable demográfica, agrupare los datos y calculare la tasa de evasión promedio de cada grupo, dado que he codificado Churn como `1` (`Sí`) y `0` (`No`), la media de la columna Churn nos dara directamente la tasa de evasión (hermoso!!).

* Utilizare `Plotly` para crear gráficos de barras que comparen visualmente estas tasas, esto permitirá identificar rápidamente qué segmentos tienen tasas de evasión más altas o más bajas que el promedio general.

**Justificación**: El análisis demográfico es un pilar en la estrategia de negocio, e identificar si ciertos grupos (por ejemplo, adultos mayores o personas sin dependientes) son más propensos a irse, permite a la empresa dirigir campañas de retención y comunicación mucho más específicas y, por lo tanto, más efectivas

In [17]:
# Lista de columnas demográficas que ya están en formato numérico/binario
demographic_cols = ['gender', 'SeniorCitizen', 'Partner', 'Dependents']

# Creamos una figura con subplots para mostrar todos los gráficos juntos
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Evasión por Género', 'Evasión por Adulto Mayor', 'Evasión por Pareja', 'Evasión por Dependientes')
)

# Mapeo de etiquetas para los gráficos
labels_map = {
    'gender': {0: 'Masculino', 1: 'Femenino'},
    'SeniorCitizen': {0: 'No', 1: 'Sí'},
    'Partner': {0: 'No', 1: 'Sí'},
    'Dependents': {0: 'No', 1: 'Sí'}
}

# Iteramos para crear cada gráfico y añadirlo a la figura
row, col = 1, 1
for column in demographic_cols:
    # Agrupamos y calculamos la tasa de churn
    churn_rate = df_final.groupby(column)['Churn'].mean().reset_index()

    # Mapeamos los valores numéricos a etiquetas de texto para mayor claridad
    churn_rate[column] = churn_rate[column].map(labels_map[column])

    # Creamos el gráfico de barras
    bar_fig = px.bar(
        churn_rate,
        x=column,
        y='Churn',
        text=churn_rate['Churn'].apply(lambda x: f'{x:.2%}'), # Formato de porcentaje
        color=column,
        color_discrete_sequence=px.colors.qualitative.Pastel
    )

    # Añadimos las barras a nuestra figura de subplots
    for trace in bar_fig.data:
        fig.add_trace(trace, row=row, col=col)

    # Actualizamos la posición para el siguiente gráfico
    col += 1
    if col > 2:
        col = 1
        row += 1

fig.update_layout(
    title_text='Análisis de Evasión por Perfil Demográfico',
    height=600,
    showlegend=False
)
fig.update_yaxes(title_text='Tasa de Evasión (Churn Rate)')
fig.show()

#### 💡 Análisis del Perfil Demográfico

Los gráficos revelan algunos patrones:

* **Género**: No existe una diferencia significativa en la tasa de evasión entre **clientes masculinos y femeninos**. Ambas tasas rondan el promedio general del `~26.5%`, por lo que este factor no parece ser un diferenciador clave.

* **Adulto Mayor (SeniorCitizen)**: Aquí encontramos la primera gran señal de alerta, ya que los adultos mayores tienen una tasa de evasión del `41.68%`, muy por encima del promedio.

* **Pareja (Partner)**: Los clientes que no tienen pareja muestran una tasa de evasión del 32.98%, mientras que aquellos con pareja tienen una tasa mucho menor (19.66%), parce que la estabilidad de tener una pareja influye en la permanencia.

* **Dependientes (Dependents)**: Los clientes sin dependientes tienen una tasa de evasión del `~31.28%`, casi el doble que la de los clientes con dependientes `~15.45%`).

***Conclusión Inicial***: He identificado un perfil demográfico de alto riesgo. Un cliente `adulto mayor`, `soltero` y `sin dependientes` es significativamente más propenso a cancelar el servicio.

### 🎯 3.4. Análisis de Evasión por Variables de Contratación

**Objetivo**: Determinar cómo las características del `contrato` y la `facturación` del cliente se relacionan con la tasa de evasión.

**Método**: Al igual que en el paso anterior, calculare la tasa de evasión promedio para cada categoría de estas variables.

**Justificación**: A diferencia de los datos demográficos, **estas variables representan decisiones y procesos que la empresa controla directamente** por lo que descubrir que un tipo de contrato o un método de pago tiene una tasa de evasión masiva es un **insight** de negocio extremadamente **accionable**, ya que puede inspirar **cambios inmediatos** en la oferta de productos, los procesos de facturación o las estrategias de fidelización.

In [18]:
# Lista de columnas relacionadas con la contratación
account_cols = ['Contract', 'PaperlessBilling', 'PaymentMethod']

# Creamos una figura con subplots
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=('Evasión por Tipo de Contrato', 'Evasión por Facturación sin Papel', 'Evasión por Método de Pago')
)

# Mapeo de etiquetas para PaperlessBilling
paperless_labels = {0: 'No', 1: 'Sí'}

# Gráfico para 'Contract'
churn_rate_contract = df_final.groupby('Contract', observed=False )['Churn'].mean().reset_index() # el valor por default para observed=False sera removido 
fig.add_trace(
    go.Bar(
        x=churn_rate_contract['Contract'],
        y=churn_rate_contract['Churn'],
        text=churn_rate_contract['Churn'].apply(lambda x: f'{x:.2%}'),
        name='Contract'
    ), row=1, col=1
)

# Gráfico para 'PaperlessBilling'
churn_rate_paperless = df_final.groupby('PaperlessBilling', observed=False)['Churn'].mean().reset_index()
churn_rate_paperless['PaperlessBilling'] = churn_rate_paperless['PaperlessBilling'].map(paperless_labels)
fig.add_trace(
    go.Bar(
        x=churn_rate_paperless['PaperlessBilling'],
        y=churn_rate_paperless['Churn'],
        text=churn_rate_paperless['Churn'].apply(lambda x: f'{x:.2%}'),
        name='PaperlessBilling'
    ), row=1, col=2
)

# Gráfico para 'PaymentMethod'
churn_rate_payment = df_final.groupby('PaymentMethod', observed=False)['Churn'].mean().reset_index()
fig.add_trace(
    go.Bar(
        x=churn_rate_payment['PaymentMethod'],
        y=churn_rate_payment['Churn'],
        text=churn_rate_payment['Churn'].apply(lambda x: f'{x:.2%}'),
        name='PaymentMethod'
    ), row=1, col=3
)

fig.update_layout(
    title_text='Análisis de Evasión por Condiciones de la Cuenta',
    height=500,
    showlegend=False
)
fig.update_yaxes(title_text='Tasa de Evasión (Churn Rate)')
fig.update_xaxes(tickangle=15) # Inclinamos las etiquetas del eje X para mejor legibilidad
fig.show()

#### 💡 Análisis de las Condiciones de Contratación

Esta visualización nos ofrece algunos de los insights más potentes hasta ahora:

* **Tipo de Contrato**: Este es, posiblemente, el factor más predictivo que he encontrado, al parecer los clientes con un contrato Mes a mes tienen una tasa de evasión del `42.71%`, una cifra astronómicamente alta en comparación con los contratos de Un año (`11.27%`) y Dos años (`2.83%`). La falta de un compromiso a largo plazo es un factor de riesgo crítico.

* **Facturación sin Papel (PaperlessBilling)**: Los clientes que optan por la facturación sin papel tienen una tasa de evasión del `33.57%`, mientras que los que reciben factura física solo tienen una tasa del `16.33%`, este resultado es contraintuitivo, podría sugerirnos que el proceso de pago en línea es confuso, que los clientes pierden la noción de sus facturas, o que este servicio es más común entre clientes menos leales.

* **Método de Pago (PaymentMethod)**: Los clientes que pagan con `Cheque electrónico` se van a una tasa del `45.26%` mientras que los clientes que usan métodos de pago automáticos como `transferencia` o `tarjeta de crédito` tienen tasas mucho más bajas. El pago manual y electrónico parece estar asociado a una menor permanencia.

***Conclusión Inicial***: Un cliente con un contrato Mes a mes, que usa Facturación sin Papel y paga con Cheque electrónico tiene, potencialmente, el mayor riesgo de evasión.



### 🎯 3.5. Análisis de Evasión por Servicios Contratados

**Objetivo**: Investigar cómo los diferentes servicios que un cliente ha contratado (tanto de telefonía como de internet) se correlacionan con su probabilidad de evasión.

**Método**: Dividire el análisis en dos partes: **servicios telefónicos** y **servicios de internet**. Para cada servicio, agrupare los datos, calculare la tasa de evasión y creare gráficos de barras para visualizar los resultados.

**Justificación**: Mi hipótesis es que los clientes con más servicios, especialmente aquellos que añaden valor y protección (como el soporte técnico o la seguridad en línea), estarán más satisfechos y, por lo tanto, serán menos propensos a irse.

#### 📞 Servicios Telefónicos

In [19]:
# Lista de columnas de servicios telefónicos
phone_cols = ['PhoneService', 'MultipleLines']

# Mapeo de etiquetas
phone_service_labels = {0: 'No', 1: 'Sí'}
multiple_lines_labels = {0: 'No', 1: 'Sí', 'No phone service': 'Sin Serv. Tel.'}


# Creamos figura con subplots
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Evasión por Tener Servicio Telefónico', 'Evasión por Múltiples Líneas')
)

# Gráfico para PhoneService
churn_rate_phone = df_final.groupby('PhoneService', observed=False)['Churn'].mean().reset_index()
churn_rate_phone['PhoneService'] = churn_rate_phone['PhoneService'].map(phone_service_labels)
fig.add_trace(go.Bar(x=churn_rate_phone['PhoneService'], y=churn_rate_phone['Churn'], text=churn_rate_phone['Churn'].apply(lambda x: f'{x:.2%}')), row=1, col=1)

# Gráfico para MultipleLines
churn_rate_lines = df_final.groupby('MultipleLines', observed=False)['Churn'].mean().reset_index()

# churn_rate_lines['MultipleLines'] = churn_rate_lines['MultipleLines'].map(multiple_lines_labels) # El mapeo ya no es necesario con la última limpieza
fig.add_trace(go.Bar(x=churn_rate_lines['MultipleLines'], y=churn_rate_lines['Churn'], text=churn_rate_lines['Churn'].apply(lambda x: f'{x:.2%}')), row=1, col=2)


fig.update_layout(title_text='Análisis de Evasión por Servicios Telefónicos', showlegend=False)
fig.update_yaxes(title_text='Tasa de Evasión (Churn Rate)')
fig.show()

#### 🛜 Servicios de Internet

In [20]:
# Lista de columnas de servicios de internet
internet_cols = [
    'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
    'TechSupport', 'StreamingTV', 'StreamingMovies'
]

# Creamos una figura con subplots para todos los gráficos de internet
fig = make_subplots(
    rows=3, cols=3,
    subplot_titles=internet_cols
)

# Iteramos para crear cada gráfico
row, col = 1, 1
for column in internet_cols:
    churn_rate = df_final.groupby(column, observed=False)['Churn'].mean().reset_index()
    bar_fig = px.bar(churn_rate, x=column, y='Churn', text=churn_rate['Churn'].apply(lambda x: f'{x:.2%}'))
    
    for trace in bar_fig.data:
        fig.add_trace(trace, row=row, col=col)
    
    col += 1
    if col > 3:
        col = 1
        row += 1

fig.update_layout(
    title_text='Análisis de Evasión por Servicios de Internet',
    height=800,
    showlegend=False
)
fig.update_yaxes(title_text='Tasa de Evasión')
fig.show()

#### 💡 Análisis de los Servicios Contratados

Este análisis es extremadamente revelador y me dio pistas muy claras sobre la retención de clientes:

##### Servicios Telefónicos:

* **Servicio Telefónico General**: No hay una diferencia notable. La gran mayoría de los clientes tiene servicio telefónico, y la tasa de evasión entre los que no lo tienen es similar.

* **Múltiples Líneas**: Los clientes con múltiples líneas tienen una tasa de evasión ligeramente más alta (`28.61%`) que aquellos con una sola línea (`25.04%`), esto es algo inesperado, pero la diferencia no es masiva.

#### Servicios de Internet:

* **Tipo de Conexión (InternetService)**: Los clientes con Fibra Óptica tienen una tasa de evasión del `41.89%`, mucho más alta que los clientes con DSL `18.96%`. Esto podría deberse a precios más altos, mayor competencia en áreas de fibra, o problemas de estabilidad del servicio a pesar de la mayor velocidad. Los clientes sin internet, como es lógico, tienen una tasa de evasión muy baja `7.40%`, ya que suelen ser clientes de solo telefonía, muy posiblemente `con contratos largos`.

* **Servicios de Protección y Soporte (Clave)**: Los clientes que NO tienen los siguientes servicios son mucho más propensos a irse:

    * **Seguridad en Línea (OnlineSecurity)**: `41.77%` de evasión sin el servicio vs. `14.61%` con el servicio.

    * **Respaldo en Línea (OnlineBackup)**: `39.93%` vs. `21.53%`.

    * **Protección de Dispositivo (DeviceProtection)**: `39.13%` vs. `22.50%`.

    * **Soporte Técnico (TechSupport)**: `41.64%` de evasión sin soporte vs. solo `15.17%` con soporte.

* **Servicios de Streaming**: Los clientes sin estos servicios tienen tasas de evasión de alrededor del `33-34%`.

***Conclusión Inicial***: Los servicios adicionales no son solo una fuente de ingresos, parece ser **herramientas de retención**, un cliente de Fibra Óptica sin Soporte Técnico y sin Seguridad en Línea presenta un perfil de riesgo de evasión extremadamente alto.

### 🎯 3.6. Análisis de Evasión por Variables Numéricas

**Objetivo**: Entender cómo las variables numéricas continuas (`antigüedad`, `cargos mensuales` y `cargos totales`) se distribuyen entre los clientes que cancelaron el servicio y los que no.

**Método**: La mejor manera de comparar distribuciones entre dos grupos son los diagramas de caja (box plots). Creare un box plot para cada variable numérica, mostrando lado a lado la distribución para los clientes que se fueron y los que se quedaron, Esto nos permitirá comparar fácilmente sus **medianas**, **rangos** y **dispersión**.

**Justificación**: Este análisis nos ayudara a responder preguntas como: **"¿Los clientes que se van suelen tener una antigüedad menor?"** o **"¿Los clientes con facturas más altas son más propensos a la evasión?"**.

In [21]:
# Lista de variables numéricas a analizar
numeric_cols = ['tenure', 'Charges.Monthly', 'Charges.Total', 'Cuentas_Diarias']

# Creamos una figura con subplots
fig = make_subplots(
    rows=1, cols=4,
    subplot_titles=('Antigüedad (meses)', 'Cargos Mensuales', 'Cargos Totales', 'Cargos Diarios')
)

# Iteramos para crear cada box plot
for i, col in enumerate(numeric_cols):
    fig.add_trace(
        go.Box(
            y=df_final[col],
            x=df_final['Churn'],
            name=col,
            boxpoints='outliers' # Muestra los valores atípicos
        ),
        row=1, col=i+1
    )

fig.update_layout(
    title_text='Distribución de Variables Numéricas por Estado de Evasión (Churn)',
    height=500,
    showlegend=False
)
fig.update_xaxes(title_text='Churn (0 = No, 1 = Sí)')
fig.show()

#### 💡 Análisis de las Variables Numéricas

Los diagramas de caja nos muestran diferencias muy claras y significativas:

* **Antigüedad (tenure)**: La **mediana** de la antigüedad de los clientes que **se quedan** es de unos `38 meses`, mientras que para los que **se van** es de solo unos `10 meses`. por lo que podria decir que los clientes nuevos son mucho más vulnerables a la evasión.

* **Cargos Mensuales (Charges.Monthly)**: Los clientes que **se van** tienden a tener **cargos mensuales** más altos `mediana de ~$80` en comparación con los que **se quedan** `mediana de ~$65`, entonces esto sumado a nuestro hallazgo sobre **la Fibra Óptica**, refuerza la idea de que los planes más caros están asociados a una mayor tasa de cancelación.

* **Cargos Totales (Charges.Total)**: Los clientes que se quedan tienen cargos totales acumulados mucho más altos, aunque esto es una consecuencia lógica de su mayor antigüedad. Lo **interesante** es que la dispersión es muy grande, indicando **una amplia variedad de perfiles de clientes leales**.

* **Cargos Diarios (Cuentas_Diarias)**: Como era de esperar, sigue el mismo patrón que los cargos mensuales. Los clientes que cancelan tienden a tener un costo diario percibido más alto.

***Conclusión Final del EDA***: Hemos identificado un perfil de cliente de alto riesgo muy consistente a través de todas nuestras variables, **clientes con poca antigüedad pero con cargos mensuales elevados, especialmente si tienen contratos mes a mes, pagan con cheque electrónico y no contratan servicios clave de soporte o seguridad**.

### 🎯 3.8. Síntesis y Creación de Perfiles de Cliente de Alto Riesgo

**Objetivo**: Sintetizar los hallazgos de todo el análisis exploratorio para construir un perfil detallado del segmento de clientes con la mayor probabilidad de evasión.

**Método**: Se identificarán las características que, de forma individual, mostraron la mayor correlación con una alta tasa de evasión, estas son:

* Tener un contrato Mes a mes.

* Tener un servicio de internet de Fibra Óptica.

* No tener contratado el servicio de Soporte Técnico.

Utilizare el filtrado de Pandas para aislar a este grupo específico de clientes que cumplen con todas estas condiciones simultáneamente, y  analizare este nuevo segmento (df_high_risk), calculare su tamaño, su tasa de evasión real, y tal vez otras características comunes.

**Justificación**: Este análisis de segmentación es la culminación de nuestro EDA, consider que en lugar de dar recomendaciones generales, me permitira identificar un arquetipo de cliente muy específico al que el negocio puede dirigir campañas de retención ultra-focalizadas.

In [22]:
# --- Primer perfil ---
high_risk_segment = df_final[
    (df_final['Contract'] == 'Month-to-month') &
    (df_final['InternetService'] == 'Fiber optic')
]

# Análisis del segmento
high_risk_count = len(high_risk_segment)
high_risk_churn_rate = high_risk_segment['Churn'].mean() * 100

print(f"--- Análisis del Segmento de Clientes de Alto Riesgo ---")
print(f"Definición: Contrato 'Mes a Mes' + Internet de 'Fibra Óptica'")
print(f"Número de clientes en este segmento: {high_risk_count}")
print(f"Tasa de evasión para este grupo: {high_risk_churn_rate:.2f}%")
print("\n" + "="*60 + "\n")


# --- Segundo perfil (CORREGIDO) ---
# Filtramos sobre el segmento que ya teníamos, buscando el valor de texto 'No'
ultra_high_risk_segment = high_risk_segment[
    high_risk_segment['TechSupport'] == 'No'  # ¡CORRECCIÓN CLAVE! Se busca el string 'No'
]

# Análisis del segmento
ultra_high_risk_count = len(ultra_high_risk_segment)
ultra_high_risk_churn_rate = ultra_high_risk_segment['Churn'].mean() * 100

print(f"--- Análisis del Segmento de Clientes de Ultra Alto Riesgo ---")
print(f"Definición: Perfil anterior + Sin 'Soporte Técnico'")
print(f"Número de clientes en este segmento: {ultra_high_risk_count}")
print(f"Tasa de evasión para este grupo: {ultra_high_risk_churn_rate:.2f}%")

# Analicemos su método de pago preferido
payment_method_risk = high_risk_segment['PaymentMethod'].value_counts(normalize=True) * 100
print("\nMétodo de pago más común en este segmento:")
print(round(payment_method_risk, 2))

--- Análisis del Segmento de Clientes de Alto Riesgo ---
Definición: Contrato 'Mes a Mes' + Internet de 'Fibra Óptica'
Número de clientes en este segmento: 2128
Tasa de evasión para este grupo: 54.61%


--- Análisis del Segmento de Clientes de Ultra Alto Riesgo ---
Definición: Perfil anterior + Sin 'Soporte Técnico'
Número de clientes en este segmento: 1796
Tasa de evasión para este grupo: 57.52%

Método de pago más común en este segmento:
PaymentMethod
Electronic check             61.42
Bank transfer (automatic)    15.37
Credit card (automatic)      13.77
Mailed check                  9.45
Name: proportion, dtype: float64


#### 💡 Conclusión Final y Perfil Definitivo

El análisis de segmentación ha revelado un perfil de cliente con una tasa de evasión alarmante, y que incluso se podria aislar mas esos perfieles son:

1. **Perfil de Alto Riesgo**: Clientes que tienen simultáneamente:

    * **Contrato**: `Month-to-month`
    * **Servicio de Internet**: `Fiber optic`
***Análisis***: Este grupo consta de `2,128` clientes, y su tasa de evasión es del `54.61%`.

2. **Perfil de Ultra Alto Riesgo**: Clientes que cumplen con este y el perfil anterior:

    * No tienen Soporte Técnico (`TechSupport` = No)

***Análisis***: Este subgrupo, aún más específico, consta de `1,796` clientes, y su tasa de evasión se eleva al `57.52%`.

#### Insight Final para el Negocio:

He identificado un segmento muy grande (`2,128` clientes) que tiene una probabilidad de más del `50%` de abandonar la compañía, y dentro de ese grupo, un subconjunto casi igual de grande (`1,796` clientes) tiene un riesgo aún mayor, por lo que una estrategia de retención dirigida a ofrecer **contratos a más largo plazo** o a incentivar la **contratación de Soporte Técnico** para este segmento podría tener un impacto masivo y medible en la reducción de la evasión general.

### 💾 Guardado de DB Final

In [23]:
# Guardar el DataFrame final en un archivo CSV
df_final.to_csv('telecom_churn_cleaned_data.csv', index=False)

## 📄 4. Informe final

### Introducción
El presente análisis se llevó a cabo con el objetivo de identificar los factores clave que impulsan la evasión de clientes (`Churn`) en **Telecom X**. La compañía enfrenta una tasa de evasión general del `26.54%`, lo que significa que más de uno de cada cuatro clientes ha optado por cancelar su servicio. Esta cifra representa una pérdida significativa de ingresos y una oportunidad crítica para mejorar la retención.

Este informe resume el proceso de análisis de datos, presenta los hallazgos más relevantes y concluye con una serie de recomendaciones estratégicas basadas en evidencia para mitigar el problema.

### 1. Limpieza y Tratamiento de Datos

Para asegurar la fiabilidad de nuestras conclusiones, se realizó un exhaustivo proceso de preparación de datos, que incluyó:

* **Extracción y Normalización**: Se cargaron los datos desde su formato original **JSON** y se transformaron de una estructura anidada a un formato tabular plano.

* **Limpieza de Datos**: Se identificaron y estandarizaron valores vacíos que estaban representados como texto. Se manejaron los registros con datos faltantes, eliminando aquellos donde la información de evasión era irrecuperable (`224` registros) e imputando otros basándonos en la lógica del negocio (ej. clientes nuevos con gasto total cero).

* **Optimización de Tipos de Datos**: Se convirtieron las columnas de texto a formatos numéricos y categóricos más eficientes, reduciendo el uso de memoria y preparando el dataset para el análisis y modelado.

* **Ingeniería de Características (Feature Engineering)**: Se creó la columna Cuentas_Diarias para proporcionar una métrica adicional sobre el comportamiento de gasto del cliente.

El resultado es un dataset robusto, limpio y coherente, listo para la analítica avanzada.

### 2. Análisis Exploratorio y Hallazgos Clave

El análisis de los datos reveló que la evasión no es un fenómeno aleatorio, sino que se concentra fuertemente en segmentos de clientes con características muy específicas.

**A.** El **Contrato** es el Factor Más Determinante

* Los clientes con contratos Mes a mes tienen una tasa de evasión del `42.71%`, mientras que esta cifra se desploma al `11.27%` para contratos de Un año y a un mínimo del `2.83%` para los de Dos años. ***La falta de un compromiso a largo plazo es el principal factor de riesgo.***

**B.** El Perfil de **Servicios** Contratados Revela Riesgos y Oportunidades

* **Internet de Fibra Óptica**: Este servicio premium está asociado a una alta tasa de evasión (`41.89%`). Esto sugiere que, a pesar de la calidad del servicio, factores como el precio o la competencia están provocando la pérdida de estos clientes.
* **Servicios de Soporte y Seguridad como Anclas de Retención**: Los servicios adicionales no son solo una fuente de ingresos, sino herramientas de retención cruciales. Clientes sin Soporte Técnico tienen una tasa de evasión del `41.64%` (vs. `15.17%` para los que sí lo tienen). De igual manera, clientes sin Seguridad en Línea se van a una tasa del `41.76%` (vs. `14.61%`).

**C.** Procesos de **Facturación** y Pagos como Puntos de Fricción

* Los clientes que pagan con Cheque electrónico presentan una alarmante tasa de evasión del `45.26%`.

* Aquellos con Facturación sin Papel también muestran una mayor propensión a irse (`33.59%` vs. `16.33%`).

Estos hallazgos sugieren que los métodos de pago manuales y los procesos de facturación digital pueden estar generando una mala experiencia de cliente.

**D.** **Síntesis**: Identificación de Segmentos de "***Ultra Alto Riesgo***"

Al combinar los factores anteriores, identificamos dos perfiles de clientes cuya probabilidad de evasión es dramáticamente alta:

* **Perfil de Alto Riesgo** (`54.61%` de Evasión): Clientes con *Contrato Mes a Mes* y servicio de *Fibra Óptica*. Este grupo representa a `2,128` clientes.

* **Perfil de Ultra Alto Riesgo** (`57.52%` de Evasión): Si al perfil anterior le sumamos la falta de *Soporte Técnico*, nos quedamos con un segmento de `1,796` clientes cuya probabilidad de abandono es más del doble de la media de la empresa.

### 3. Conclusiones e Insights

**El `Churn` está Concentrado**: La pérdida de clientes no es generalizada, sino que está impulsada por perfiles de clientes claramente identificables y predecibles.

**La Flexibilidad Mata la Lealtad**: El contrato mes a mes, si bien es atractivo para los clientes, es el principal vehículo de la evasión.

**Los Servicios Adicionales son Herramientas de Retención**: Servicios como el *Soporte Técnico* y la *Seguridad en Línea* crean "pegajosidad" (**stickiness**), haciendo que los clientes estén más integrados y satisfechos con el ecosistema de la empresa.

**La Experiencia de Pago es Crítica**: Un proceso de pago engorroso o poco fiable (como el cheque electrónico) es un fuerte repelente de clientes.

### 4. Recomendaciones Estratégicas

Basado en los hallazgos, se proponen las siguientes acciones para reducir la tasa de evasión:

* Implementar **Campañas** de Migración de Contrato:

    * *Acción*: Diseñar y ejecutar campañas de **marketing proactivas dirigidas al "Perfil de Alto Riesgo"** para incentivarlos a *migrar de un contrato Mes a Mes a uno de Un Año*, ofreciendo un descuento en la tarifa mensual o un servicio adicional gratuito durante los primeros meses, empresas como Canva y Microsoft las promocionan constantemente.

* Crear **Paquetes** de "Retención y Protección":

    * *Acción*: Para los clientes de *Fibra Óptica* (especialmente los nuevos), **empaquetar de forma atractiva el Soporte Técnico y la Seguridad en Línea**. Se podría ofrecer el primer año de estos servicios con un descuento significativo para aumentar su adopción y, por ende, la retención.

* Optimizar la **Experiencia de Pago**:

    * *Acción*: Investigar y mejorar el flujo de pago con Cheque electrónico y el sistema de Facturación sin Papel. Simultáneamente, **crear incentivos** (ej. un pequeño descuento único) para que los clientes cambien a **métodos de pago automáticos** y más fiables como la **domiciliación** bancaria o la tarjeta de crédito.

* Desarrollar un **Modelo Predictivo** de Churn:

    * *Acción*: Utilizar este informe como la base para que el equipo de Data Science construya un modelo de machine learning, este modelo podrá asignar una "**puntuación de riesgo de evasión**" a cada cliente mensualmente, permitiendo al equipo de retención actuar de forma preventiva y personalizada antes de que el cliente decida irse.